In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
# #          print(os.path.join(dirname, filename))


## Prediction :

In [2]:


import os
import cv2
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

import matplotlib.pyplot as plt


from keras.layers import Dense, Conv2D,MaxPooling2D,Flatten,InputLayer,AveragePooling2D
from keras.layers import BatchNormalization,Dropout,Activation



from tensorflow.keras.utils import to_categorical
from tqdm import tqdm  # For progress bar






import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tensorflow.keras.models import load_model




/opt/conda/lib/python3.10/site-packages/pydicom/data/data_manager.py:375: UserWarning: A download failure occurred while attempting to retrieve US1_J2KR.dcm
  warn_and_log(
/opt/conda/lib/python3.10/site-packages/pydicom/data/data_manager.py:375: UserWarning: A download failure occurred while attempting to retrieve MR-SIEMENS-DICOM-WithOverlays.dcm
  warn_and_log(
/opt/conda/lib/python3.10/site-packages/pydicom/data/data_manager.py:375: UserWarning: A download failure occurred while attempting to retrieve OBXXXX1A.dcm
  warn_and_log(
/opt/conda/lib/python3.10/site-packages/pydicom/data/data_manager.py:375: UserWarning: A download failure occurred while attempting to retrieve US1_UNCR.dcm
  warn_and_log(
/opt/conda/lib/python3.10/site-packages/pydicom/data/data_manager.py:375: UserWarning: A download failure occurred while attempting to retrieve color3d_jpeg_baseline.dcm
  warn_and_log(


In [3]:
data_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv'

data = pd.read_csv(data_path)  # This should contain study_id and the target column
columns = data.columns
columns
for column in columns[1:]:
    print(str(column))

spinal_canal_stenosis_l1_l2
spinal_canal_stenosis_l2_l3
spinal_canal_stenosis_l3_l4
spinal_canal_stenosis_l4_l5
spinal_canal_stenosis_l5_s1
left_neural_foraminal_narrowing_l1_l2
left_neural_foraminal_narrowing_l2_l3
left_neural_foraminal_narrowing_l3_l4
left_neural_foraminal_narrowing_l4_l5
left_neural_foraminal_narrowing_l5_s1
right_neural_foraminal_narrowing_l1_l2
right_neural_foraminal_narrowing_l2_l3
right_neural_foraminal_narrowing_l3_l4
right_neural_foraminal_narrowing_l4_l5
right_neural_foraminal_narrowing_l5_s1
left_subarticular_stenosis_l1_l2
left_subarticular_stenosis_l2_l3
left_subarticular_stenosis_l3_l4
left_subarticular_stenosis_l4_l5
left_subarticular_stenosis_l5_s1
right_subarticular_stenosis_l1_l2
right_subarticular_stenosis_l2_l3
right_subarticular_stenosis_l3_l4
right_subarticular_stenosis_l4_l5
right_subarticular_stenosis_l5_s1


## to get an overview about our dataset

In [4]:


# Load and inspect the directions DataFrame
df_directions = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_series_descriptions.csv')

print("Shape of df_directions:", df_directions.shape)
print("First few rows of df_directions:")
print(df_directions.head())

# Check unique study_ids and series_ids
unique_study_ids = df_directions['study_id'].unique()
unique_series_ids = df_directions['series_id'].unique()
print("Unique Study IDs in df_directions:", unique_study_ids)
print("Unique Series IDs in df_directions:", unique_series_ids)



Shape of df_directions: (3, 3)
First few rows of df_directions:
   study_id   series_id series_description
0  44036939  2828203845        Sagittal T1
1  44036939  3481971518           Axial T2
2  44036939  3844393089   Sagittal T2/STIR
Unique Study IDs in df_directions: [44036939]
Unique Series IDs in df_directions: [2828203845 3481971518 3844393089]


In [5]:
train_csv_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv'

# Load the CSV file into a DataFrame
df_train_check = pd.read_csv(train_csv_path)
csv_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv'
df_check = pd.read_csv(csv_path)

# Filter rows where 'series_description' contains 'Sagittal T2/STIR' (case-insensitive)
df_Sagittal_T2_STIR = df_check[df_check['series_description'].str.contains('Sagittal T2/STIR', case=False, na=False)]
df_Sagittal_T2_STIR
label_csv_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv'

# Read the CSV file
label_df0 = pd.read_csv(label_csv_path)
label_df1=pd.merge(label_df0, df_Sagittal_T2_STIR, on=['study_id', 'series_id'], how='inner')
label_df1['condition'].unique()

array(['Spinal Canal Stenosis'], dtype=object)

It shows that for this direction all conditions for outputs are 'Spinal Canal Stenosis'. So, there is a corelation=1 for 'Sagittal_T2_STIR' direction and condition=Spinal Canal Stenosis. In order to predict the condition for Spinal Canal Stenosis just all the images inside folder related to Sagittal_T2_STIR condition will be used.

In [6]:
df_check = pd.read_csv(csv_path)
# Filter rows where 'series_description' contains 'Sagittal T2/STIR' (case-insensitive)
df_Sagittal_T1 = df_check[df_check['series_description'].str.contains('Sagittal T1', case=False, na=False)]
df_Sagittal_T1
# Read the CSV file
label_df0 = pd.read_csv(label_csv_path)

label_df1=pd.merge(label_df0, df_Sagittal_T1, on=['study_id', 'series_id'], how='inner')
label_df1['condition'].unique()

array(['Right Neural Foraminal Narrowing',
       'Left Neural Foraminal Narrowing', 'Spinal Canal Stenosis'],
      dtype=object)

It shows that for predicting the condition for"Right Neural Foraminal Narrowing', 'Left Neural Foraminal Narrowing', 'Spinal Canal Stenosis' images with the direction=Sagittal T1 should be used

In [7]:
df_check = pd.read_csv(csv_path)

# Filter rows where 'series_description' contains 'Sagittal T2/STIR' (case-insensitive)
df_Axial_T2 = df_check[df_check['series_description'].str.contains('Axial T2', case=False, na=False)]

label_df1=pd.merge(label_df0, df_Axial_T2, on=['study_id', 'series_id'], how='inner')
label_df1['condition'].unique()

array(['Left Subarticular Stenosis', 'Right Subarticular Stenosis'],
      dtype=object)

it shows that in order to predict condition='Left Subarticular Stenosis', 'Right Subarticular Stenosis' images with direction=Axial_T2 must be used.

Lets take a look at our data in the test folder. All data inside this folder belongs to one person. three folders just show different angles that MRI images are captured. Let's take a look at photoes inside one siries_id for this patient.

In [8]:
# import os
# import cv2
# import pydicom
# import numpy as np
# import matplotlib.pyplot as plt
# from pydicom.pixel_data_handlers.util import apply_voi_lut

# def read_dicom_image(file_path, voi_lut=True, fix_monochrome=True):
#     dicom = pydicom.dcmread(file_path)
    
#     if voi_lut:
#         data = apply_voi_lut(dicom.pixel_array, dicom)
#     else:
#         data = dicom.pixel_array

#     if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
#         data = np.amax(data) - data

#     # Normalize the pixel values between 0 and 1
#     data = data - np.min(data)
#     data = data / np.max(data)

#     # Convert to 8-bit grayscale (0-255 range)
#     return (data * 255).astype(np.uint8)

# def enhance_image(image):
#     # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) for better contrast
#     clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
#     image = clahe.apply(image)

#     # Optionally, apply Gaussian Blurring to reduce noise
#     image = cv2.GaussianBlur(image, (5, 5), 0)

#     return image

# def show_two_dicom_images(study_id_1, series_id_1, instance_number_1, 
#                           study_id_2, series_id_2, instance_number_2):
#     # Base path to the images
#     image_base_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/'
    
#     # Construct the file paths for both images
#     series_path_1 = os.path.join(image_base_path, str(study_id_1), str(series_id_1))
#     file_path_1 = os.path.join(series_path_1, f"{instance_number_1}.dcm")
    
#     series_path_2 = os.path.join(image_base_path, str(study_id_2), str(series_id_2))
#     file_path_2 = os.path.join(series_path_2, f"{instance_number_2}.dcm")
    
#     if not os.path.exists(file_path_1):
#         print(f"File {file_path_1} does not exist.")
#         return
    
#     if not os.path.exists(file_path_2):
#         print(f"File {file_path_2} does not exist.")
#         return
    
#     try:
#         # Read and process both DICOM images
#         image1 = read_dicom_image(file_path_1)
#         image2 = read_dicom_image(file_path_2)
        
#         # Enhance the images using CLAHE and Gaussian Blurring
#         enhanced_image1 = enhance_image(image1)
#         enhanced_image2 = enhance_image(image2)
        
#         # Plot and display the two images side by side using matplotlib
#         fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        
#         # Display the first image
#         axes[0].imshow(enhanced_image1, cmap='gray')
#         axes[0].set_title(f'Study: {study_id_1}, Series: {series_id_1}, Instance: {instance_number_1}')
#         axes[0].axis('off')  # Hide axis labels
        
#         # Display the second image
#         axes[1].imshow(enhanced_image2, cmap='gray')
#         axes[1].set_title(f'Study: {study_id_2}, Series: {series_id_2}, Instance: {instance_number_2}')
#         axes[1].axis('off')  # Hide axis labels
        
#         plt.tight_layout()
#         plt.show()
    
#     except Exception as e:
#         print(f"Error processing files {file_path_1} and/or {file_path_2}: {e}")

# # Example usage:
# study_id_1 = 44036939
# series_id_1 = 2828203845
# instance_number_1 = 20


# study_id_2 = 44036939
# series_id_2 =2828203845
# instance_number_2 = 12

# show_two_dicom_images(study_id_1, series_id_1, instance_number_1, 
#                       study_id_2, series_id_2, instance_number_2)

as can be seen in the above photoes. both photoes are for one patient. the resolution of photoes are different. It is clear that as the MRI photo shows the spinal cord better, the better our model can predict the result. But we are not going to check it personally. We should make a machine that reads all the images one by one and predicts the result for each photo. Then it should keep the result of the image that our model with higher certainty predicts its category. "seviere - normal- moderate".

In [9]:


import pandas as pd

# File paths
data_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv'
df_csv_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv'

# Read the CSV files
df = pd.read_csv(df_csv_path)
data = pd.read_csv(data_path )



 
columns=data.columns


In [10]:

loaded_models = {}

# Loop through the columns and dynamically load the models
for column in columns[1:]:  # Skipping 'study_id'
    # Construct the model file path dynamically based on the column name
    model_name = f'/kaggle/input/_{column}.keras/keras/default/1/_{column}.keras'
    
    print(f'Loading model: {model_name}')
    
    # Load the model and store it in the dictionary with the column name as the key
    loaded_models[column] = load_model(model_name)

# Now you have all the models loaded in the `loaded_models` dictionary, 
# accessible by their respective column names.
print("All models loaded successfully!")

Loading model: /kaggle/input/_spinal_canal_stenosis_l1_l2.keras/keras/default/1/_spinal_canal_stenosis_l1_l2.keras
Loading model: /kaggle/input/_spinal_canal_stenosis_l2_l3.keras/keras/default/1/_spinal_canal_stenosis_l2_l3.keras
Loading model: /kaggle/input/_spinal_canal_stenosis_l3_l4.keras/keras/default/1/_spinal_canal_stenosis_l3_l4.keras
Loading model: /kaggle/input/_spinal_canal_stenosis_l4_l5.keras/keras/default/1/_spinal_canal_stenosis_l4_l5.keras
Loading model: /kaggle/input/_spinal_canal_stenosis_l5_s1.keras/keras/default/1/_spinal_canal_stenosis_l5_s1.keras
Loading model: /kaggle/input/_left_neural_foraminal_narrowing_l1_l2.keras/keras/default/1/_left_neural_foraminal_narrowing_l1_l2.keras
Loading model: /kaggle/input/_left_neural_foraminal_narrowing_l2_l3.keras/keras/default/1/_left_neural_foraminal_narrowing_l2_l3.keras
Loading model: /kaggle/input/_left_neural_foraminal_narrowing_l3_l4.keras/keras/default/1/_left_neural_foraminal_narrowing_l3_l4.keras
Loading model: /kagg

In [11]:


import os
import pandas as pd
import numpy as np
import cv2
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tensorflow.keras.models import load_model
# Get direction from df_directions based on study_id and series_id
def get_image_direction(study_id, series_id, df_directions):
    direction_row = df_directions[
        (df_directions['study_id'] == study_id) & 
        (df_directions['series_id'] == series_id)
    ]
    if not direction_row.empty:
        return direction_row.iloc[0]['series_description']
    else:
        return None
# Load directions data (update the path to your directions CSV)
df_directions = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_series_descriptions.csv')

# Convert series_id and study_id to string for consistent data handling
df_directions['study_id'] = df_directions['study_id'].astype(str).str.strip()
df_directions['series_id'] = df_directions['series_id'].astype(str).str.strip()
df_directions



,study_id,series_id,series_description
0,44036939,2828203845,Sagittal T1
1,44036939,3481971518,Axial T2
2,44036939,3844393089,Sagittal T2/STIR


In [12]:
import os
import pydicom
import numpy as np
import cv2
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Function to read and preprocess the DICOM image
def read_dicom_image(file_path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.dcmread(file_path)
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    return (data * 255).astype(np.uint8)

# Preprocess image for prediction
def preprocess_image(image):
    image = cv2.resize(image, (50, 50), interpolation=cv2.INTER_AREA)
    image = np.stack((image,) * 3, axis=-1)
    image = image.astype('float32') / 255
    image = np.expand_dims(image, axis=0)
    return image

def get_study_and_series_ids(image_path):
    parts = image_path.split(os.sep)  # Split the path using the directory separator
    study_id = parts[-2]  # Second-to-last folder is the study_id
    series_id = parts[-3]   # Third-to-last folder is the series_id
    return study_id, series_id

# Get direction from df_directions based on study_id and series_id
def get_image_direction(study_id, series_id, df_directions):
    direction_row = df_directions[
        (df_directions['study_id'] == study_id) & 
        (df_directions['series_id'] == series_id)
    ]
    if not direction_row.empty:
        return direction_row.iloc[0]['series_description']
    else:
        return None

# Predict based on direction and models
def predict_based_on_direction(image_path, df_directions, results_df):
    # Get the study_id and series_id from the image path
    series_id, study_id = get_study_and_series_ids(image_path)
    
    # Get the direction from the CSV using the study_id and series_id
    direction = get_image_direction(study_id, series_id, df_directions)
    
    if direction is None:
        print("Direction not found for study_id:", study_id, "series_id:", series_id)
        return results_df
    
    # Preprocess the image
    image = read_dicom_image(image_path)
    preprocessed_image = preprocess_image(image)
    
    # Category mapping for results
    category_mapping = {1: 'Normal/Mild', 2: 'Moderate', 4: 'Severe'}
    
    # Initialize model keys to an empty list
    model_keys = []
    
    # Determine the models based on the direction
    if direction == 'Sagittal T2/STIR':
        # Predict using Sagittal T2/STIR models
        model_keys = ['spinal_canal_stenosis_l1_l2', 'spinal_canal_stenosis_l2_l3',
                      'spinal_canal_stenosis_l3_l4', 'spinal_canal_stenosis_l4_l5',
                      'spinal_canal_stenosis_l5_s1']
    elif direction == 'Sagittal T1':
        # Predict using Sagittal T1 models only
        model_keys = [
            'spinal_canal_stenosis_l1_l2', 'spinal_canal_stenosis_l2_l3',
            'spinal_canal_stenosis_l3_l4', 'spinal_canal_stenosis_l4_l5',
            'spinal_canal_stenosis_l5_s1', 
            'left_neural_foraminal_narrowing_l1_l2',
            'left_neural_foraminal_narrowing_l2_l3', 
            'left_neural_foraminal_narrowing_l3_l4',
            'left_neural_foraminal_narrowing_l4_l5', 
            'left_neural_foraminal_narrowing_l5_s1',
            'right_neural_foraminal_narrowing_l1_l2', 
            'right_neural_foraminal_narrowing_l2_l3',
            'right_neural_foraminal_narrowing_l3_l4', 
            'right_neural_foraminal_narrowing_l4_l5',
            'right_neural_foraminal_narrowing_l5_s1']
    elif direction == 'Axial T2':
        # Predict using Axial T2 models
        model_keys = ['left_subarticular_stenosis_l1_l2', 'left_subarticular_stenosis_l2_l3',
                      'left_subarticular_stenosis_l3_l4', 'left_subarticular_stenosis_l4_l5',
                      'left_subarticular_stenosis_l5_s1', 'right_subarticular_stenosis_l1_l2',
                      'right_subarticular_stenosis_l2_l3', 'right_subarticular_stenosis_l3_l4',
                      'right_subarticular_stenosis_l4_l5', 'right_subarticular_stenosis_l5_s1']
    else:
        print("Unknown direction:", direction)
        return results_df
    
    # Iterate over the relevant models for the direction and predict
    for model_key in model_keys:
        model = loaded_models[model_key]
        predictions = model.predict(preprocessed_image, verbose=0)
        predictions = predictions / np.sum(predictions)  # Normalize the predictions
        
        # Add predictions to results_df
        image_id = f"{study_id}_{model_key}"
        predicted_category = np.argmax(predictions, axis=1)[0] + 1
        predicted_label = category_mapping.get(predicted_category, 'Unknown')
        
        new_row = {
            "row_id": image_id,
            "normal_mild": predictions[0][0],
            "moderate": predictions[0][1],
            "severe": predictions[0][2]
        }
        
        # Check if new_row has valid data before concatenating
        if not all(value is None for value in new_row.values()):  # Ensure no empty or all-NA entries
            results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)
    
    return results_df

def result_higher_probability(direcrory_path):
    # Loop through all DICOM files in the specified directory
    dataframes_list = []
    for file_name in os.listdir(direcrory_path):  # Use the parameter 'direcrory_path' correctly
        if file_name.endswith('.dcm'):
            image_path = os.path.join(direcrory_path, file_name)  # Use the parameter 'direcrory_path'
            print(image_path)
            
            results_df = pd.DataFrame(columns=["row_id", "normal_mild", "moderate", "severe"])
            results_df = predict_based_on_direction(image_path, df_directions, results_df)
            dataframes_list.append(results_df)

    for i in range(len(dataframes_list[0])):
        # Get the current row for comparison
        row_id = dataframes_list[0].iloc[i]['row_id']  # Assuming row_id is the same across DataFrames

        # Initialize a flag to determine if this row should be kept
        keep_row = True

        # Get the row values from the first DataFrame for comparison
        base_row_values = dataframes_list[0].iloc[i, 1:].values  # Ignore 'row_id'

        # Compare with other DataFrames
        for df in dataframes_list[1:]:
            compare_row_values = df.iloc[i, 1:].values  # Ignore 'row_id'

            # Check if the base row values are all higher than the current DataFrame row
            if not all(base_row_values > compare_row_values):
                keep_row = False
                break  # Exit the loop if any value is not higher

        # If the row should be kept, add it to results_df
        if keep_row:
            results_df = results_df.append(dataframes_list[0].iloc[i], ignore_index=True)

    # Display the resulting DataFrame
    print("DataFrame with all higher values:")
    print(results_df)
    return results_df

# Example usage
# direcrory_path = create_full_paths(base_directory, series)






def get_series_list(base_directory):
    # Get the list of directories within the base_directory
    series = [name for name in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, name))]
    return series
def create_full_paths(base_directory, series):
    # Create full paths for each directory in the series list
    full_paths = [os.path.join(base_directory, name) for name in series]
    return full_paths

# Path to the base directory
base_directory = r"/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/"
series = get_series_list(base_directory)
series_full_paths = create_full_paths(base_directory, series)

#

final_dataframes = []

# Loop through each path in series_full_paths
for path in series_full_paths:
    print(f"Processing directory: {path}")
    # Call the function for each directory and store the resulting DataFrame
    final_df = result_higher_probability(path)
    
    # Append the resulting DataFrame to the list
    final_dataframes.append(final_df)

Processing directory: /kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/12.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/18.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/9.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/22.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/25.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/14.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/11.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/24.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-

In [13]:


df1=final_dataframes[0]
df2=final_dataframes[1]
df3=final_dataframes[2]
combined_df = pd.concat([df1, df2, df3], ignore_index=True)
combined_df['max_value'] = combined_df[['normal_mild', 'moderate', 'severe']].max(axis=1)
combined_df = combined_df.sort_values(by=['row_id', 'max_value'], ascending=[True, False])
final_df = combined_df.drop_duplicates(subset='row_id', keep='first')
final_df = final_df.drop(columns=['max_value'])
final_df



,row_id,normal_mild,moderate,severe
20,44036939_left_neural_foraminal_narrowing_l1_l2,2.494336e-05,0.999975,1.761967e-08
21,44036939_left_neural_foraminal_narrowing_l2_l3,1.587500e-01,0.841089,1.605747e-04
22,44036939_left_neural_foraminal_narrowing_l3_l4,2.574144e-01,0.742582,3.378703e-06
23,44036939_left_neural_foraminal_narrowing_l4_l5,9.739483e-01,0.025759,2.927832e-04
24,44036939_left_neural_foraminal_narrowing_l5_s1,9.988702e-01,0.000992,1.374624e-04
5,44036939_left_subarticular_stenosis_l1_l2,1.534797e-03,0.998332,1.335757e-04
6,44036939_left_subarticular_stenosis_l2_l3,1.809981e-03,0.998174,1.636619e-05
7,44036939_left_subarticular_stenosis_l3_l4,8.289886e-01,0.170624,3.877662e-04
8,44036939_left_subarticular_stenosis_l4_l5,3.251429e-03,0.007050,9.896982e-01
9,44036939_left_subarticular_stenosis_l5_s1,3.457263e-01,0.653348,9.259604e-04


In [14]:


desired_order = [
    "44036939_left_neural_foraminal_narrowing_l1_l2",
    "44036939_left_neural_foraminal_narrowing_l2_l3",
    "44036939_left_neural_foraminal_narrowing_l3_l4",
    "44036939_left_neural_foraminal_narrowing_l4_l5",
    "44036939_left_neural_foraminal_narrowing_l5_s1",
    "44036939_left_subarticular_stenosis_l1_l2",
    "44036939_left_subarticular_stenosis_l2_l3",
    "44036939_left_subarticular_stenosis_l3_l4",
    "44036939_left_subarticular_stenosis_l4_l5",
    "44036939_left_subarticular_stenosis_l5_s1",
    "44036939_right_neural_foraminal_narrowing_l1_l2",
    "44036939_right_neural_foraminal_narrowing_l2_l3",
    "44036939_right_neural_foraminal_narrowing_l3_l4",
    "44036939_right_neural_foraminal_narrowing_l4_l5",
    "44036939_right_neural_foraminal_narrowing_l5_s1",
    "44036939_right_subarticular_stenosis_l1_l2",
    "44036939_right_subarticular_stenosis_l2_l3",
    "44036939_right_subarticular_stenosis_l3_l4",
    "44036939_right_subarticular_stenosis_l4_l5",
    "44036939_right_subarticular_stenosis_l5_s1",
    "44036939_spinal_canal_stenosis_l1_l2",
    "44036939_spinal_canal_stenosis_l2_l3",
    "44036939_spinal_canal_stenosis_l3_l4",
    "44036939_spinal_canal_stenosis_l4_l5",
    "44036939_spinal_canal_stenosis_l5_s1"
]


final_df['row_id'] = pd.Categorical(final_df['row_id'], categories=desired_order, ordered=True)



In [15]:
final_df=final_df.reset_index(drop=True)
final_df
# final_df.to_csv("/kaggle/working/submission.csv", index=False)
pd.set_option('display.float_format', lambda x: '%.16f' % x)
final_df.to_csv("/kaggle/working/submission.csv", index=False, float_format='%.16f')
final_df

,row_id,normal_mild,moderate,severe
0,44036939_left_neural_foraminal_narrowing_l1_l2,0.0000249433614954,0.9999750852584839,0.0000000176196675
1,44036939_left_neural_foraminal_narrowing_l2_l3,0.1587500274181366,0.8410894274711609,0.0001605747092981
2,44036939_left_neural_foraminal_narrowing_l3_l4,0.2574144005775452,0.7425822019577026,0.0000033787032407
3,44036939_left_neural_foraminal_narrowing_l4_l5,0.9739482998847961,0.0257588978856802,0.0002927831956185
4,44036939_left_neural_foraminal_narrowing_l5_s1,0.9988701939582825,0.0009923513280228,0.0001374624407617
5,44036939_left_subarticular_stenosis_l1_l2,0.0015347969019786,0.9983316063880920,0.0001335757260676
6,44036939_left_subarticular_stenosis_l2_l3,0.0018099807202816,0.9981736540794373,0.0000163661879924
7,44036939_left_subarticular_stenosis_l3_l4,0.8289886116981506,0.1706235855817795,0.0003877661947627
8,44036939_left_subarticular_stenosis_l4_l5,0.0032514287158847,0.0070503819733858,0.9896982312202454
9,44036939_left_subarticular_stenosis_l5_s1,0.3457262814044952,0.6533477902412415,0.0009259604266845
